In [1]:
# Library imports
import pandas as pd
import numpy as np
import json

In [2]:
# Turn off the SettingWithCopyWarning (do this post-dev only)
pd.set_option('mode.chained_assignment', None)

In [3]:
# Read the datasets
df1 = pd.read_csv('../data/primary_dataset.csv')
df2 = pd.read_csv('../data/secondary_dataset.csv')

## Primary dataset processing

In [4]:
df1.columns

Index(['Rank', 'Title', 'Genre', 'Description', 'Director', 'Actors', 'Year',
       'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)',
       'Metascore'],
      dtype='object')

In [5]:
# Convert movie, director and actor names to lower case (for convenience)
for col_name in ['Title', 'Director', 'Actors']:
    df1[col_name] = df1[col_name].map(str.lower)
# Convert to a python list
df1['Actors List'] = df1.Actors.map(lambda x: [i.strip() for i in x.split(",")])

In [6]:
# Keep only relevant columns (for now - add more fields for extra filter features)
df1_relevant = df1[['Title', 'Actors List']]
# Standardize column names
df1_relevant.columns = ['title', 'actors']
df1_relevant.head()

,title,actors
0,guardians of the galaxy,"[chris pratt, vin diesel, bradley cooper, zoe ..."
1,prometheus,"[noomi rapace, logan marshall-green, michael f..."
2,split,"[james mcavoy, anya taylor-joy, haley lu richa..."
3,sing,"[matthew mcconaughey, reese witherspoon, seth ..."
4,suicide squad,"[will smith, jared leto, margot robbie, viola ..."


## Secondary dataset processing

In [7]:
df2.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [8]:
# We have the "\xa0" character at the end ofthe titles: we want to get rid of it
df2.movie_title = df2.movie_title.replace(u'\xa0', u'', regex=True)
# Convert movie, director and actor names to lower case (for convenience)
for col_name in ['movie_title', 'director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name']:
    df2[col_name] = df2[col_name].apply(str)
    df2[col_name] = df2[col_name].map(str.lower)
# Combine columns and convert to python list
df2['actors_list'] = df2[['actor_1_name', 'actor_2_name', 'actor_3_name']].apply(lambda x: ','.join(x.dropna()), axis=1)
df2['actors_list'] = df2['actors_list'].map(lambda x: [i.strip() for i in x.split(",")])

In [9]:
# Keep only relevant columns (for now - add more fields for extra filter features)
df2_relevant = df2[['movie_title', 'director_name', 'actors_list', 'title_year', 'imdb_score', 'gross', 'budget']]
# Drop rows with NaN values
df2_relevant = df2_relevant.dropna()
# Convert title year (which is in float) to integer
df2_relevant['title_year'] = df2_relevant['title_year'].apply(int)
# Standardize column names
df2_relevant.columns = ['title', 'director', 'actors', 'year', 'imdb_rating', 'revenue', 'budget']
df2_relevant.head()

,title,director,actors,year,imdb_rating,revenue,budget
0,avatar,james cameron,"[cch pounder, joel david moore, wes studi]",2009,7.9,760505847.0,237000000.0
1,pirates of the caribbean: at world's end,gore verbinski,"[johnny depp, orlando bloom, jack davenport]",2007,7.1,309404152.0,300000000.0
2,spectre,sam mendes,"[christoph waltz, rory kinnear, stephanie sigman]",2015,6.8,200074175.0,245000000.0
3,the dark knight rises,christopher nolan,"[tom hardy, christian bale, joseph gordon-levitt]",2012,8.5,448130642.0,250000000.0
5,john carter,andrew stanton,"[daryl sabara, samantha morton, polly walker]",2012,6.6,73058679.0,263700000.0


## Combine datasets

**Pointers about combining datasets:**
- Keep only df2 entries because it has budget values
- Fetch actors from df1 for more actors in common entries

In [10]:
print('df1:', df1_relevant.shape)
print('df2:', df2_relevant.shape)

df1: (1000, 2)
df2: (3891, 7)


In [11]:
# Merge the two dataframes to keep the common entries
df_merge = df2_relevant.merge(df1_relevant, on = "title")
df_merge.head()

,title,director,actors_x,year,imdb_rating,revenue,budget,actors_y
0,avatar,james cameron,"[cch pounder, joel david moore, wes studi]",2009,7.9,760505847.0,237000000.0,"[sam worthington, zoe saldana, sigourney weave..."
1,pirates of the caribbean: at world's end,gore verbinski,"[johnny depp, orlando bloom, jack davenport]",2007,7.1,309404152.0,300000000.0,"[johnny depp, orlando bloom, keira knightley, ..."
2,spectre,sam mendes,"[christoph waltz, rory kinnear, stephanie sigman]",2015,6.8,200074175.0,245000000.0,"[daniel craig, christoph waltz, léa seydoux, r..."
3,the dark knight rises,christopher nolan,"[tom hardy, christian bale, joseph gordon-levitt]",2012,8.5,448130642.0,250000000.0,"[christian bale, tom hardy, anne hathaway, gar..."
4,john carter,andrew stanton,"[daryl sabara, samantha morton, polly walker]",2012,6.6,73058679.0,263700000.0,"[taylor kitsch, lynn collins, willem dafoe, sa..."


In [12]:
# Concatenate the actors list from the primary dataset (run this cell only once)
df_merge['actors_x'] = df_merge['actors_x'] + df_merge['actors_y']
df_merge = df_merge.drop(columns = ['actors_y'])

In [13]:
# Standardize column names
df_merge.columns = ['title', 'director', 'actors', 'year', 'imdb_rating', 'revenue', 'budget']
# Keep only the distinct actor names
df_merge['actors'] = df_merge['actors'].apply(lambda x: list(set(x)))
df_merge.head()

,title,director,actors,year,imdb_rating,revenue,budget
0,avatar,james cameron,"[cch pounder, sam worthington, joel david moor...",2009,7.9,760505847.0,237000000.0
1,pirates of the caribbean: at world's end,gore verbinski,"[johnny depp, keira knightley, geoffrey rush, ...",2007,7.1,309404152.0,300000000.0
2,spectre,sam mendes,"[daniel craig, léa seydoux, stephanie sigman, ...",2015,6.8,200074175.0,245000000.0
3,the dark knight rises,christopher nolan,"[tom hardy, anne hathaway, gary oldman, christ...",2012,8.5,448130642.0,250000000.0
4,john carter,andrew stanton,"[samantha morton, polly walker, lynn collins, ...",2012,6.6,73058679.0,263700000.0


In [14]:
# Get the remaining secondary dataset entries
df2_exclusive = df2_relevant[~df2_relevant['title'].isin(df_merge['title'])]
# Finally, combine the two dataframes
df_comb = df_merge.append(df2_exclusive)
df_comb.head()

,title,director,actors,year,imdb_rating,revenue,budget
0,avatar,james cameron,"[cch pounder, sam worthington, joel david moor...",2009,7.9,760505847.0,237000000.0
1,pirates of the caribbean: at world's end,gore verbinski,"[johnny depp, keira knightley, geoffrey rush, ...",2007,7.1,309404152.0,300000000.0
2,spectre,sam mendes,"[daniel craig, léa seydoux, stephanie sigman, ...",2015,6.8,200074175.0,245000000.0
3,the dark knight rises,christopher nolan,"[tom hardy, anne hathaway, gary oldman, christ...",2012,8.5,448130642.0,250000000.0
4,john carter,andrew stanton,"[samantha morton, polly walker, lynn collins, ...",2012,6.6,73058679.0,263700000.0


In [15]:
print('common_1n2:', df_merge.shape)
print('exclusive2:', df2_exclusive.shape)
print('combine1n2:', df_comb.shape)

common_1n2: (651, 7)
exclusive2: (3242, 7)
combine1n2: (3893, 7)


In [16]:
actor_counts = df_merge['actors'].apply(len)
actor_counts.value_counts()

6    275
5    220
7    115
4     41
Name: actors, dtype: int64

In [17]:
# df_comb.to_csv(r'tmp1.csv', index = False)

## Create director-actor pairs

In [18]:
# Seperate the actors list into multiple rows
df_comb = df_comb.explode('actors')
# Remove duplicate entries
df_comb = df_comb.drop_duplicates()
df_comb.head()

,title,director,actors,year,imdb_rating,revenue,budget
0,avatar,james cameron,cch pounder,2009,7.9,760505847.0,237000000.0
0,avatar,james cameron,sam worthington,2009,7.9,760505847.0,237000000.0
0,avatar,james cameron,joel david moore,2009,7.9,760505847.0,237000000.0
0,avatar,james cameron,zoe saldana,2009,7.9,760505847.0,237000000.0
0,avatar,james cameron,sigourney weaver,2009,7.9,760505847.0,237000000.0


In [19]:
# Group by director-actor pair, keep the remaining columns as lists
df_group = df_comb.groupby(['director', 'actors'], as_index=False)[['title', 'year', 'imdb_rating', 'revenue', 'budget']].agg(lambda x: list(x))
df_group.head()

,director,actors,title,year,imdb_rating,revenue,budget
0,aaron schneider,bill cobbs,[get low],[2009],[7.1],[9176553.0],[7500000.0]
1,aaron schneider,bill murray,[get low],[2009],[7.1],[9176553.0],[7500000.0]
2,aaron schneider,robert duvall,[get low],[2009],[7.1],[9176553.0],[7500000.0]
3,aaron seltzer,alyson hannigan,[date movie],[2006],[2.7],[48546578.0],[20000000.0]
4,aaron seltzer,carmen electra,[date movie],[2006],[2.7],[48546578.0],[20000000.0]


In [20]:
# Get the movies count for each pair
df_group['movies_count'] = df_group['title'].apply(len)
# Keep only the pairs with at least 2 movies
df_filter = df_group[df_group.movies_count > 1]
df_filter.head()

,director,actors,title,year,imdb_rating,revenue,budget,movies_count
29,adam mckay,john c. reilly,"[talladega nights: the ballad of ricky bobby, ...","[2006, 2008]","[6.6, 6.9]","[148213377.0, 100468793.0]","[73000000.0, 65000000.0]",2
36,adam mckay,steve carell,"[the big short, anchorman 2: the legend contin...","[2015, 2013, 2004]","[7.8, 6.3, 7.2]","[70235322.0, 2175312.0, 84136909.0]","[28000000.0, 50000000.0, 26000000.0]",3
37,adam mckay,will ferrell,"[the other guys, talladega nights: the ballad ...","[2010, 2006, 2008, 2013, 2004]","[6.7, 6.6, 6.9, 6.3, 7.2]","[119219978.0, 148213377.0, 100468793.0, 217531...","[100000000.0, 73000000.0, 65000000.0, 50000000...",5
197,alejandro g. iñárritu,naomi watts,[birdman or (the unexpected virtue of ignoranc...,"[2014, 2003]","[7.8, 7.7]","[42335698.0, 16248701.0]","[18000000.0, 20000000.0]",2
226,alex kendrick,alex kendrick,"[courageous, facing the giants]","[2011, 2006]","[7.0, 6.7]","[34522221.0, 10174663.0]","[2000000.0, 100000.0]",2


In [21]:
print('orig_pairs:', df_group.shape)
print('filt_pairs:', df_filter.shape)

orig_pairs: (12316, 8)
filt_pairs: (510, 8)


## Handle troublesome cases

#### People paired with themselves

In [22]:
# Find self-paired people (we want different people in a pair)
df_filter['same_pairs'] = np.where((df_filter.director == df_filter.actors), True, False)
print('Number of such people:', df_filter[df_filter['same_pairs'] == True].shape[0])

# Remove such rows with self-paired people
df_filter = df_filter[df_filter['same_pairs'] == False]

Number of such people: 19


#### Directors who have acted too

In [23]:
# Create a list of directors and actors
directors = df_filter['director'].drop_duplicates()
actors = df_filter['actors'].drop_duplicates()

In [24]:
# Directors who acted too / Actors who directed too (same thing)
acting_directors = list(set(directors[directors.isin(actors)]))
# directing_actors = list(set(actors[actors.isin(directors)]))
print('Number of such people:', len(acting_directors))

# Remove such rows where directors acted (if not removed, same person belongs to both groups)
df_filter = df_filter[~df_filter['actors'].isin(acting_directors)]

Number of such people: 4


In [25]:
# df_filter.to_csv(r'tmp2.csv', index = False)

## Final result stored in JSON file

In [26]:
# Create a final dataframe with the relevant columns
df_final = df_filter.drop(columns = ['same_pairs'])
df_final.head()

,director,actors,title,year,imdb_rating,revenue,budget,movies_count
29,adam mckay,john c. reilly,"[talladega nights: the ballad of ricky bobby, ...","[2006, 2008]","[6.6, 6.9]","[148213377.0, 100468793.0]","[73000000.0, 65000000.0]",2
36,adam mckay,steve carell,"[the big short, anchorman 2: the legend contin...","[2015, 2013, 2004]","[7.8, 6.3, 7.2]","[70235322.0, 2175312.0, 84136909.0]","[28000000.0, 50000000.0, 26000000.0]",3
37,adam mckay,will ferrell,"[the other guys, talladega nights: the ballad ...","[2010, 2006, 2008, 2013, 2004]","[6.7, 6.6, 6.9, 6.3, 7.2]","[119219978.0, 148213377.0, 100468793.0, 217531...","[100000000.0, 73000000.0, 65000000.0, 50000000...",5
197,alejandro g. iñárritu,naomi watts,[birdman or (the unexpected virtue of ignoranc...,"[2014, 2003]","[7.8, 7.7]","[42335698.0, 16248701.0]","[18000000.0, 20000000.0]",2
228,alex kendrick,erin bethea,"[fireproof, facing the giants]","[2008, 2006]","[6.5, 6.7]","[33451479.0, 10174663.0]","[500000.0, 100000.0]",2


In [27]:
print('final_count:', df_final.shape)

final_count: (487, 8)


In [28]:
# df_final.to_csv(r'tmp3.csv', index = False)

In [29]:
### Store the result into a json file
directors_list = df_final['director'].drop_duplicates()
actors_list = df_final['actors'].drop_duplicates()

# Iterate rows:
nodes = list()
names = list()
for director in directors_list:
    nodes.append({'id': director, 'group' : 1})
    names.append(director)
for actor in actors_list:
    nodes.append({'id': actor, 'group' : 2})
    names.append(actor)

links = list()
for index, row in df_final.iterrows():
    links.append({"source": row['director'], "target": row['actors'], "title": row['title'], "year": row['year'],
                  "imdb_rating": row['imdb_rating'], "revenue": row['revenue'], "budget": row['budget'], 
                  "counts": row['movies_count']})

json_file = {
    'nodes': nodes,
    'links': links
}

with open('new-result.json', 'w') as fp:
    json.dump(json_file, fp)